In [1]:
import random
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import seaborn as sns
import numpy as np
import pickle
from joblib import Parallel, delayed
from math import log2, ceil
import ardent
# from scipy.ndimage import zoom
import time
import pandas as pd

from proglearn.progressive_learner import ProgressiveLearner
from proglearn.deciders import SimpleArgmaxAverage
from proglearn.transformers import TreeClassificationTransformer, NeuralClassificationTransformer
from proglearn.voters import TreeClassificationVoter, KNNClassificationVoter
# from proglearn.sims import generate_gaussian_parity

In [2]:
from sklearn.datasets import make_blobs


def _generate_2d_rotation(theta=0):
    R = np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])

    return R


def generate_gaussian_parity(
    n_samples,
    centers=None,
    class_label=None,
    cluster_std=0.25,
    angle_params=None,
    random_state=None,
):

    if random_state != None:
        np.random.seed(random_state)

    if centers == None:
        centers = np.array([(-0.5, 0.5), (0.5, 0.5), (-0.5, -0.5), (0.5, -0.5)])

    if class_label == None:
        class_label = [0, 1, 1, 0]

    blob_num = len(class_label)

    # get the number of samples in each blob with equal probability
    samples_per_blob = np.random.multinomial(
        n_samples, 1 / blob_num * np.ones(blob_num)
    )

    X, y = make_blobs(
        n_samples=samples_per_blob,
        n_features=2,
        centers=centers,
        cluster_std=cluster_std,
    )

    for blob in range(blob_num):
        y[np.where(y == blob)] = class_label[blob]

    if angle_params != None:
        R = _generate_2d_rotation(angle_params)
        X = X @ R

    return X, y

In [3]:
def to_grid(X_task):
    h = 0.01
    x_min, x_max = X_task[:,0].min()-0.1, X_task[:,0].max()+0.1
    y_min, y_max = X_task[:,1].min()-0.1, X_task[:,1].max()+0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    new_X_task = np.c_[xx.ravel(), yy.ravel()]
    return xx, yy, new_X_task

In [4]:
def init_forest(max_depth):
    default_transformer_class = TreeClassificationTransformer
    default_transformer_kwargs = {"kwargs" : {"max_depth" : max_depth}}

    default_voter_class = TreeClassificationVoter
    default_voter_kwargs = {}

    default_decider_class = SimpleArgmaxAverage
    default_decider_kwargs = {"classes" : np.arange(2)}
    progressive_learner = ProgressiveLearner(default_transformer_class = default_transformer_class,
                                            default_transformer_kwargs = default_transformer_kwargs,
                                            default_voter_class = default_voter_class,
                                            default_voter_kwargs = default_voter_kwargs,
                                            default_decider_class = default_decider_class,
                                            default_decider_kwargs = default_decider_kwargs)
    uf = ProgressiveLearner(default_transformer_class = default_transformer_class,
                                            default_transformer_kwargs = default_transformer_kwargs,
                                            default_voter_class = default_voter_class,
                                            default_voter_kwargs = default_voter_kwargs,
                                            default_decider_class = default_decider_class,
                                            default_decider_kwargs = default_decider_kwargs)
    return progressive_learner, uf

In [5]:
seed_no = 3

In [6]:
np.random.seed(seed_no)
#source data
X_task1, y_task1 = generate_gaussian_parity(100, angle_params=0)
test_task1, test_label_task1 = generate_gaussian_parity(1000, angle_params=0)

#target data
X_task2, y_task2 = generate_gaussian_parity(100, angle_params=np.pi/4)
test_task2, test_label_task2 = generate_gaussian_parity(1000, angle_params=np.pi/4)

In [7]:
def init_forest(max_depth):
    default_transformer_class = TreeClassificationTransformer
    default_transformer_kwargs = {"kwargs" : {"max_depth" : max_depth}}

    default_voter_class = TreeClassificationVoter
    default_voter_kwargs = {}

    default_decider_class = SimpleArgmaxAverage
    default_decider_kwargs = {"classes" : np.arange(2)}
    progressive_learner = ProgressiveLearner(default_transformer_class = default_transformer_class,
                                            default_transformer_kwargs = default_transformer_kwargs,
                                            default_voter_class = default_voter_class,
                                            default_voter_kwargs = default_voter_kwargs,
                                            default_decider_class = default_decider_class,
                                            default_decider_kwargs = default_decider_kwargs)
    uf = ProgressiveLearner(default_transformer_class = default_transformer_class,
                                            default_transformer_kwargs = default_transformer_kwargs,
                                            default_voter_class = default_voter_class,
                                            default_voter_kwargs = default_voter_kwargs,
                                            default_decider_class = default_decider_class,
                                            default_decider_kwargs = default_decider_kwargs)
    return progressive_learner, uf

In [8]:
progressive_learner, uf = init_forest(7)

In [9]:
n_trees = 10
np.random.seed(seed_no)

progressive_learner.add_task(X_task1, y_task1, num_transformers=n_trees)
progressive_learner.add_task(X_task2, y_task2, num_transformers=n_trees)

In [10]:
# L2F o lddmm

np.random.seed(seed_no)
xx1, yy1, test_task1_grid = to_grid(test_task1)
# task1_pos1 = progressive_learner.predict(test_task1_grid, task_id=0)
task1_pos1, task1_pos2 = progressive_learner.predict(test_task1_grid, task_id=0, registration=True)

xx2, yy2, test_task2_grid = to_grid(test_task2)
# task2_pos2 = progressive_learner.predict(test_task2_grid, task_id=0)
task2_pos1, task2_pos2 = progressive_learner.predict(test_task2_grid, task_id=1, registration=True)

In [11]:
# uf

np.random.seed(seed_no)
uf.add_task(X_task1, y_task1, num_transformers=2*n_trees)
uf.add_task(X_task2, y_task2, num_transformers=2*n_trees)
xx1, yy1, test_task1_grid = to_grid(test_task1)
uf_task1 = uf.predict(test_task1, transformer_ids=[0], task_id=0)

xx2, yy2, test_task2_grid = to_grid(test_task2)
uf_task2 = uf.predict(test_task2, transformer_ids=[1], task_id=1)

In [12]:
def grid_to_pred(task_pos_reshaped, test_task, xx, yy):
    pred = np.empty((len(test_task), ))
    for i in range(len(test_task)):
        x_ind = np.where(test_task[i,0] <= xx[0,:])[0][0]
        y_ind = np.where(test_task[i,1] <= yy[:,0])[0][0]
        pred[i] = task_pos_reshaped[y_ind, x_ind]  # y_ind row, x_ind column
    pred = pred.reshape((-1,1))
    pred = np.hstack((pred, 1-pred))
    pred = np.argmax(pred, axis=1)

    return pred

In [13]:
def grid_to_score(l2f_task, test_task, xx, yy):
    pred = np.empty((len(test_task), ))
    l2f_task = l2f_task.reshape(xx.shape)
    for i in range(len(test_task)):
        x_ind = np.where(test_task[i,0] <= xx[0,:])[0][0]
        y_ind = np.where(test_task[i,1] <= yy[:,0])[0][0]
        pred[i] = l2f_task[x_ind, y_ind]
    # pred = pred.reshape((1,-1))

    return pred

In [14]:
task1_pos1_reshaped = task1_pos1[:,0].reshape(xx1.shape)
task1_pos2_reshaped = task1_pos2[:,0].reshape(xx1.shape)
task2_pos1_reshaped = task2_pos1[:,0].reshape(xx2.shape)
task2_pos2_reshaped = task2_pos2[:,0].reshape(xx2.shape)

In [16]:
def lddmm_reg(in_task_pos_reshaped, cross_task_pos_reshaped):
    print("start running lddmm")
    transform = ardent.Transform()
    reference = in_task_pos_reshaped
    moving = cross_task_pos_reshaped

    try:
        transform.register(target=moving, template=reference)
    except RuntimeError:
        try:
            transform.register(target=moving, template=reference, affine_stepsize=0.2)
        except RuntimeError:
            try:
                transform.register(target=moving, template=reference, affine_stepsize=0.1)
            except RuntimeError:
                try:
                    transform.register(target=moving, template=reference, affine_stepsize=0.05)
                except RuntimeError:
                    return moving

    deformed_moving = transform.transform_image(
        subject=moving,
        output_shape=moving.shape,
        deform_to='template')
    print("finish running lddmm")
    return deformed_moving

In [17]:
errors = np.zeros(6, dtype=float)

In [18]:
vote0 = np.mean([task1_pos1_reshaped, task1_pos2_reshaped], axis=0)
errors[0] = 1 - np.mean(grid_to_pred(vote0, test_task1, xx1, yy1) == test_label_task1)  # l2f_task1_error
vote1 = np.mean([task2_pos1_reshaped, task2_pos2_reshaped], axis=0)
errors[1] = 1 - np.mean(grid_to_pred(vote1, test_task2, xx2, yy2) == test_label_task2)  # l2f_task2_error

In [19]:
errors

array([0.083, 0.079, 0.   , 0.   , 0.   , 0.   ])

In [20]:
start_time = time.time()
task1_pos2_deformed = lddmm_reg(task1_pos1_reshaped, task1_pos2_reshaped)
vote2 = np.mean([task1_pos1_reshaped, task1_pos2_deformed], axis=0)
errors[2] = 1 - np.mean(grid_to_pred(vote2, test_task1, xx1, yy1) == test_label_task1)  # l2f_lddmm_task1_error
task2_pos1_deformed = lddmm_reg(task2_pos2_reshaped, task2_pos1_reshaped)
vote3 = np.mean([task2_pos2_reshaped, task2_pos1_deformed], axis=0)
errors[3] = 1 - np.mean(grid_to_pred(vote3, test_task2, xx2, yy2) == test_label_task2)  # l2f_lddmm_task2_error

print("--- %s seconds ---" % (time.time() - start_time))

start running lddmm
finish running lddmm
start running lddmm
finish running lddmm
--- 314.81873202323914 seconds ---


In [21]:
errors[4] = 1 - np.mean(uf_task1 == test_label_task1)  # uf_task1_error
errors[5] = 1 - np.mean(uf_task2 == test_label_task2)  # uf_task1_error

In [22]:
errors

array([0.083, 0.079, 0.074, 0.082, 0.081, 0.091])

In [23]:
def exp(n_task1, n_task2, n_test=1000, task1_angle=0, task2_angle=np.pi/4, n_trees=10, rand=None):
    if rand is not None:
        np.random.seed(rand)
    progressive_learner, uf = init_forest(max_depth=ceil(log2(n_task1)))
    errors = np.zeros(6, dtype=float)

    # np.random.seed(rand)
    #source data
    X_task1, y_task1 = generate_gaussian_parity(n_task1, angle_params=task1_angle)
    test_task1, test_label_task1 = generate_gaussian_parity(n_test, angle_params=task1_angle)
    #target data
    X_task2, y_task2 = generate_gaussian_parity(n_task2, angle_params=task2_angle)
    test_task2, test_label_task2 = generate_gaussian_parity(n_test, angle_params=task2_angle)

    # np.random.seed(rand)
    # l2f
    progressive_learner.add_task(X_task1, y_task1, num_transformers=n_trees)
    progressive_learner.add_task(X_task2, y_task2, num_transformers=n_trees)

    xx1, yy1, test_task1_grid = to_grid(test_task1)
    task1_pos1, task1_pos2 = progressive_learner.predict(test_task1_grid, task_id=0, registration=True)
    xx2, yy2, test_task2_grid = to_grid(test_task2)
    task2_pos1, task2_pos2 = progressive_learner.predict(test_task2_grid, task_id=1, registration=True)

    task1_pos1_reshaped = task1_pos1[:,0].reshape(xx1.shape)
    task1_pos2_reshaped = task1_pos2[:,0].reshape(xx1.shape)
    task2_pos1_reshaped = task2_pos1[:,0].reshape(xx2.shape)
    task2_pos2_reshaped = task2_pos2[:,0].reshape(xx2.shape)

    vote0 = np.mean([task1_pos1_reshaped, task1_pos2_reshaped], axis=0)
    errors[0] = 1 - np.mean(grid_to_pred(vote0, test_task1, xx1, yy1) == test_label_task1)  # l2f_task1_error
    vote1 = np.mean([task2_pos1_reshaped, task2_pos2_reshaped], axis=0)
    errors[1] = 1 - np.mean(grid_to_pred(vote1, test_task2, xx2, yy2) == test_label_task2)  # l2f_task2_error

    # np.random.seed(rand)
    # lddmm
    task1_pos2_deformed = lddmm_reg(task1_pos1_reshaped, task1_pos2_reshaped)
    vote2 = np.mean([task1_pos1_reshaped, task1_pos2_deformed], axis=0)
    errors[2] = 1 - np.mean(grid_to_pred(vote2, test_task1, xx1, yy1) == test_label_task1)  # l2f_lddmm_task1_error
    task2_pos1_deformed = lddmm_reg(task2_pos2_reshaped, task2_pos1_reshaped)
    vote3 = np.mean([task2_pos2_reshaped, task2_pos1_deformed], axis=0)
    errors[3] = 1 - np.mean(grid_to_pred(vote3, test_task2, xx2, yy2) == test_label_task2)  # l2f_lddmm_task2_error


    # uf
    uf.add_task(X_task1, y_task1, num_transformers=2*n_trees)
    uf.add_task(X_task2, y_task2, num_transformers=2*n_trees)

    uf_task1 = uf.predict(test_task1, transformer_ids=[0], task_id=0)
    uf_task2 = uf.predict(test_task2, transformer_ids=[1], task_id=1)

    errors[4] = 1 - np.mean(uf_task1 == test_label_task1)  # uf_task1_error
    errors[5] = 1 - np.mean(uf_task2 == test_label_task2)  # uf_task1_error

    return errors

In [ ]:
start_time = time.time()
# errors = exp(100, 100)
mc_rep = 10
# name the set of errors as error_ntask1_ntask2_frac_rep (rotation_param = np.pi/frac)
error_100_100_4_3 = np.array(
        Parallel(n_jobs=-1,verbose=2)(
        delayed(exp)(n_task1=100, n_task2=100) for _ in range(mc_rep)
      )
    )

print("--- %s seconds ---" % (time.time() - start_time))